# Weather Underground stopped allowing access to historical data, and with the 10 calls/minute limit were slow to use. This is how weather data is pulled from DarkSky.net

In [1]:
import requests
import time
import json
import config
import datetime
import pandas as pd
import os

In [3]:
 #Chicago coordinates
latitude = "41.8781"
longitude = "-87.6298"


### Getting Historical Weather Data

In [16]:
enddate = datetime.datetime(2017,12,31) #final date to pull weather for.

startdate = enddate - datetime.timedelta(days = 1)

2017-12-31 00:00:00


In [70]:
print(int(startdate.timestamp())) #UNIX time format for URL request

1514613600


In [48]:
day_datetime = startdate
ctr = 0
responses = [] #list to store responses from API
while day_datetime <= enddate: #Need to change dates and call <1000/day to get weather for all dates.
    dayiter = str(int(day_datetime.timestamp())) #string date format for current date on loop.
    print(dayiter)
    ctr = ctr + 1
    url = str("https://api.darksky.net/forecast/"+ config.api_k + "/"+ latitude + 
           "," + longitude + ","+ str(dayiter) + "?exclude=currently,minutely,flags,hourly")
    
    response = requests.get(url).json()
    responses.append(response)
    day_datetime = day_datetime + datetime.timedelta(days = 1)
    
    if ctr == 4: #Purely during testing, to prevent an infinite loop of calls and resultant API ban.
        break

                            

1514613600
1514700000
[{'latitude': 41.8781, 'longitude': -87.6298, 'timezone': 'America/Chicago', 'daily': {'data': [{'time': 1514613600, 'summary': 'Mostly cloudy overnight.', 'icon': 'partly-cloudy-night', 'sunriseTime': 1514639952, 'sunsetTime': 1514672980, 'moonPhase': 0.41, 'precipIntensity': 0, 'precipIntensityMax': 0, 'precipProbability': 0, 'temperatureHigh': 7.38, 'temperatureHighTime': 1514638800, 'temperatureLow': 1.35, 'temperatureLowTime': 1514696400, 'apparentTemperatureHigh': -10.19, 'apparentTemperatureHighTime': 1514667600, 'apparentTemperatureLow': -15.58, 'apparentTemperatureLowTime': 1514689200, 'dewPoint': -2.97, 'humidity': 0.64, 'pressure': 1027.59, 'windSpeed': 13.24, 'windBearing': 296, 'cloudCover': 0.22, 'visibility': 9.26, 'temperatureMin': 1.35, 'temperatureMinTime': 1514696400, 'temperatureMax': 14.85, 'temperatureMaxTime': 1514613600, 'apparentTemperatureMin': -15.58, 'apparentTemperatureMinTime': 1514689200, 'apparentTemperatureMax': 3.87, 'apparentTemp

In [49]:
print(response) #Sample response

{'latitude': 41.8781, 'longitude': -87.6298, 'timezone': 'America/Chicago', 'daily': {'data': [{'time': 1514700000, 'summary': 'Partly cloudy in the morning.', 'icon': 'partly-cloudy-night', 'sunriseTime': 1514726360, 'sunsetTime': 1514759429, 'moonPhase': 0.45, 'precipIntensity': 0, 'precipIntensityMax': 0, 'precipProbability': 0, 'temperatureHigh': 13.81, 'temperatureHighTime': 1514754000, 'temperatureLow': -6.16, 'temperatureLowTime': 1514815200, 'apparentTemperatureHigh': -0.05, 'apparentTemperatureHighTime': 1514750400, 'apparentTemperatureLow': -25.86, 'apparentTemperatureLowTime': 1514811600, 'dewPoint': -3.69, 'humidity': 0.63, 'pressure': 1034.8, 'windSpeed': 10.15, 'windBearing': 294, 'cloudCover': 0.25, 'visibility': 9.91, 'temperatureMin': 1.47, 'temperatureMinTime': 1514700000, 'temperatureMax': 13.81, 'temperatureMaxTime': 1514754000, 'apparentTemperatureMin': -14.52, 'apparentTemperatureMinTime': 1514700000, 'apparentTemperatureMax': -0.05, 'apparentTemperatureMaxTime': 

### Store historic weather data in DataFrame, to write to csv and be used to train model. Full script will do this in batches to stay within call limit.

In [68]:
datlist = []
for resp in responses:
    
    datlist.append({"Date": datetime.datetime.fromtimestamp(resp["daily"]["data"][0]["time"]).strftime("%m/%d/%Y"),
                       "MeanAppTemp": (resp["daily"]["data"][0]["apparentTemperatureLow"] +
                                      resp["daily"]["data"][0]["apparentTemperatureHigh"])/2,
                       "PrecipProb": resp["daily"]["data"][0]["precipProbability"],
                       "PrecipIntensity": resp["daily"]["data"][0]["precipProbability"]})
weatherDat = pd.DataFrame(datlist)
weatherDat


,Date,MeanAppTemp,PrecipIntensity,PrecipProb
0,12/30/2017,-12.885,0,0
1,12/31/2017,-12.955,0,0


### Get current forecast for next few days (to be calculated at regular intervals - every few hours - and stored,                                         and used in predictions for the web app). Only the immediate next day is used.

In [4]:
#Pure forecast
url = str("https://api.darksky.net/forecast/"+ config.api_k + "/"+ latitude + 
           "," + longitude + "?exclude=currently,minutely,flags,hourly")
    
response = requests.get(url).json()
print(response)

{'latitude': 41.8781, 'longitude': -87.6298, 'timezone': 'America/Chicago', 'daily': {'summary': 'Mixed precipitation on Friday and next Sunday, with temperatures falling to 35°F on Tuesday.', 'icon': 'snow', 'data': [{'time': 1521349200, 'summary': 'Partly cloudy starting in the afternoon, continuing until evening.', 'icon': 'partly-cloudy-day', 'sunriseTime': 1521374285, 'sunsetTime': 1521417688, 'moonPhase': 0.04, 'precipIntensity': 0.0001, 'precipIntensityMax': 0.002, 'precipIntensityMaxTime': 1521421200, 'precipProbability': 0.13, 'precipType': 'rain', 'temperatureHigh': 54.63, 'temperatureHighTime': 1521410400, 'temperatureLow': 34.28, 'temperatureLowTime': 1521464400, 'apparentTemperatureHigh': 54.63, 'apparentTemperatureHighTime': 1521410400, 'apparentTemperatureLow': 25.24, 'apparentTemperatureLowTime': 1521464400, 'dewPoint': 23.81, 'humidity': 0.53, 'pressure': 1016.88, 'windSpeed': 0.78, 'windGust': 7.03, 'windGustTime': 1521432000, 'windBearing': 164, 'cloudCover': 0.12, '

In [5]:
datlist = []
resp = response
for j in range(4):  #For today and next 3 days
    datlist.append({"Date": datetime.datetime.fromtimestamp(resp["daily"]["data"][j]["time"]).strftime("%m/%d/%Y"),
                       "MeanAppTemp": (resp["daily"]["data"][j]["apparentTemperatureLow"] +
                                      resp["daily"]["data"][j]["apparentTemperatureHigh"])/2,
                       "PrecipProb": resp["daily"]["data"][j]["precipProbability"],
                       "PrecipIntensity": resp["daily"]["data"][j]["precipIntensity"]})
weatherForecast = pd.DataFrame(datlist)
weatherForecast

,Date,MeanAppTemp,PrecipIntensity,PrecipProb
0,03/18/2018,39.935,0.0001,0.13
1,03/19/2018,22.460,0.0002,0.12
2,03/20/2018,21.510,0.0001,0.12
3,03/21/2018,24.725,0.0001,0.11


### Write data and store (On AWS only the small forecast would be repeatedly stored and overwritten)

In [85]:
curNBdir = !echo %cd%
curNBdir #No good way to use relative path in Jupyter.

['C:\\Users\\varun\\Documents\\Northwestern\\Studies\\Winter2018\\Analytics_Value_Chain\\Varun_Project\\MSiA423_WebApp\\Develop\\msia423_project_model\\notebooks']

In [86]:
weatherDat.to_csv("historicalWeatherSample.csv", index = False) #Obviously in Py file save to data directory
weatherForecast.to_csv("currentForecast.csv", index = False)